## 載入套件

In [ ]:
!pip install fake-useragent
import pandas as pd
from time import sleep
import datetime
from time import sleep
import random
from pandas import DataFrame
import requests as rs
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import os
from glob import glob
import shutil
import numpy as np

## 資料取得

#### Ubike站點資料

In [ ]:
def task():
  url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
  data = pd.read_json(url)
  datatime = (datetime.datetime.utcnow()+datetime.timedelta(hours=8)).strftime("%Y%m%d_%H%M")
  data[data["sarea"] == "臺大專區"].to_csv('ubikeData_'+datatime+".csv", encoding = 'utf_8_sig', index= 0)

#### 腳踏車拖吊資料

In [ ]:
regionA = ['小福', '農業陳列館', '普通大樓', '化工系館', '校史室', '新體育館', '文學院', '凝態中心', '海洋所', 
           '大門', 'A區其他', '化學系', '新生大樓', '博雅館', '思亮館', '天文數學館', '舊體育館', '日文系', 
           '舊哲學系', '土木系館', '樂學館', '電機一館', '原分所', '生化所', '數學研究中心', '新月台', '網球場', '全球變遷中心']
regionB = ['電機二館', '獸醫系館', '總圖書館', '計算機中心', '志鴻館', '女八九宿舍', '社會系館', '第一活動中心', 
           '博理館', '機械系', '農藝館', 'Ｂ區其他', '資工所', '霖澤館', '國發所', '新聞所', '生機二館', '應力所', 
           '心理系', '漁科所', '語言中心', '萬才館', '國青宿舍', '綜合教室', '圖資系', '工綜館', '鄭江樓']
regionC=['1號館', '行政大樓', '共同教室', '研一、大一女舍', '小小福', '鹿鳴廣場', '舟山路', '管理學院', '生命科學館', 
         '農化系', '地理系', '華南銀行', 'Ｃ區其他', '行政大樓門口', '女一舍門口', '農綜館', '園藝系', '森林系', '大氣系', 
         '物理系館', '學生心理輔導中心', '地質系', '華南銀行前停車場', '駐警隊', '食科所', '女生1235宿舍', '卓越聯合中心']
regionD = ['Ｄ區其他', '知武館', '明達館', '水工所', '航測館', '中非大樓', '工科海洋', '環工所', '基隆路156巷', 
           '生物產業機電學系', '保健中心', '農機館', '男七舍', '男五舍', '男一舍', '男六舍', '男八舍', '男三舍', '台大動物醫院']
regionE = ['校外捷運公館站', '水源校區', '新生南路沿線及巷弄', '復興南路沿線及巷弄', '辛亥路沿線及巷弄', '科技大樓周邊']

ua = UserAgent()
user_agent = ua.random
headers = {'user-agent': user_agent}

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
def towTask():
  # setting attributes
  crawlDate = (datetime.datetime.utcnow()+datetime.timedelta(hours=8)).strftime("%Y%m%d")
  crawlDateForPD = (datetime.datetime.utcnow()+datetime.timedelta(hours=8)).date()

  count = 1
  dateLst = []
  timeLst = []
  amPm = []
  placeLst = []
  region = []
  stoping_condition = False

  for i in range(25):
      if stoping_condition:
        break
      print(i)
      try:
          url = "https://mybike.ntu.edu.tw/bike.announcement/towing/page/" + str(i)
          res = rs.get(url, headers=headers)
      except:
          break
      sleep(random.randint(1, 3))
      soup = BeautifulSoup(res.text, 'lxml')
      

      for i in soup.select('body > div.pages > div > div > div > div.row > div.col-12.col-md-9 > ul'):
          texts = i.text
          texts = texts.split('\n')
          texts = list(filter(lambda a: a != '', texts))

          for j in range(0, len(texts), 4):
              placeLst.append(texts[j + 2])
              tempTime = texts[j + 3]
              tempTime = tempTime.split(' ')
              date = tempTime[0]
              times = tempTime[1]
              ampm = tempTime[2]              
              # check whether day we want
              if date[8:10] != crawlDate[6:8]: # change matching criteria
                stoping_condition = True
              else:
                dateLst.append(date)
                timeLst.append(times)
                amPm.append(ampm) 
                if texts[j + 2] in regionA:
                    region.append('A')
                elif texts[j + 2] in regionB:
                    region.append('B')
                elif texts[j + 2] in regionC:
                    region.append('C')
                elif texts[j + 2] in regionD:
                    region.append('D')
                else:
                    region.append('E')

  # turn into DataFrame
  bikeData = pd.DataFrame(list(zip(region, placeLst, dateLst, timeLst, amPm)), 
                          columns = ['region', 'place', 'date', 'time', 'AM/PM'])
  bikeData = bikeData.set_index('region')
  if os.path.exists('towData_'+crawlDate+".csv"):
    # put new in file
    oldData = pd.read_csv('towData_'+crawlDate+".csv")
    oldData = oldData.set_index('region')
    bikeData = bikeData.append(oldData)
    #print(bikeData)
    # drop duplicate
    bikeData = bikeData.drop_duplicates()
    print('append on old file')
    print(bikeData)
  else:
    # create new
    print('create new file')

  # create/replace with new file
  bikeData.to_csv('towData_'+crawlDate+".csv")

#### 取得資料

In [ ]:
def run():   
  if datetime.datetime.now().minute in {00, 15, 30, 45}:  
    task()
    towTask()
  else: 
    sleep(60)

In [ ]:
while True:
  run()

## Preprocessing

In [ ]:
def timerange(df): #assign time range
  if (df["time"] >= "00:00:00") and (df["time"] < "00:15:00"):
    return '0000'
  elif (df["time"] >= "00:15:00") and (df["time"] < "00:30:00"):
    return '0015'
  elif (df["time"] >= "00:30:00") and (df["time"] < "00:45:00"):
    return '0030'
  elif (df["time"] >= "00:45:00") and (df["time"] < "01:00:00"):
    return '0045'
  elif (df["time"] >= "01:00:00") and (df["time"] < "01:15:00"):
    return '0100'
  elif (df["time"] >= "01:15:00") and (df["time"] < "01:30:00"):
    return '0115'
  elif (df["time"] >= "01:30:00") and (df["time"] < "01:45:00"):
    return '0130'
  elif (df["time"] >= "01:45:00") and (df["time"] < "02:00:00"):
    return '0145'
  elif (df["time"] >= "02:00:00") and (df["time"] < "02:15:00"):
    return '0200'
  elif (df["time"] >= "02:15:00") and (df["time"] < "02:30:00"):
    return '0215'
  elif (df["time"] >= "02:30:00") and (df["time"] < "02:45:00"):
    return '0230'
  elif (df["time"] >= "02:45:00") and (df["time"] < "03:00:00"):
    return '0245'
  elif (df["time"] >= "03:00:00") and (df["time"] < "03:15:00"):
    return '0300'
  elif (df["time"] >= "03:15:00") and (df["time"] < "03:30:00"):
    return '0315'
  elif (df["time"] >= "03:30:00") and (df["time"] < "03:45:00"):
    return '0330'
  elif (df["time"] >= "03:45:00") and (df["time"] < "04:00:00"):
    return '0345'
  elif (df["time"] >= "04:00:00") and (df["time"] < "04:15:00"):
    return '0400'
  elif (df["time"] >= "04:15:00") and (df["time"] < "04:30:00"):
    return '0415'
  elif (df["time"] >= "04:30:00") and (df["time"] < "04:45:00"):
    return '0430'
  elif (df["time"] >= "04:45:00") and (df["time"] < "05:00:00"):
    return '0445'
  elif (df["time"] >= "05:00:00") and (df["time"] < "05:15:00"):
    return '0500'
  elif (df["time"] >= "05:15:00") and (df["time"] < "05:30:00"):
    return '0515'
  elif (df["time"] >= "05:30:00") and (df["time"] < "05:45:00"):
    return '0530'
  elif (df["time"] >= "05:45:00") and (df["time"] < "06:00:00"):
    return '0545'
  elif (df["time"] >= "06:00:00") and (df["time"] < "06:15:00"):
    return '0600'
  elif (df["time"] >= "06:15:00") and (df["time"] < "06:30:00"):
    return '0615'
  elif (df["time"] >= "06:30:00") and (df["time"] < "06:45:00"):
    return '0630'
  elif (df["time"] >= "06:45:00") and (df["time"] < "07:00:00"):
    return '0645'
  elif (df["time"] >= "07:00:00") and (df["time"] < "07:15:00"):
    return '0700'
  elif (df["time"] >= "07:15:00") and (df["time"] < "07:30:00"):
    return '0715'
  elif (df["time"] >= "07:30:00") and (df["time"] < "07:45:00"):
    return '0730'
  elif (df["time"] >= "07:45:00") and (df["time"] < "08:00:00"):
    return '0745'
  elif (df["time"] >= "08:00:00") and (df["time"] < "08:15:00"):
    return '0800'
  elif (df["time"] >= "08:15:00") and (df["time"] < "08:30:00"):
    return '0815'
  elif (df["time"] >= "08:30:00") and (df["time"] < "08:45:00"):
    return '0830'
  elif (df["time"] >= "08:45:00") and (df["time"] < "09:00:00"):
    return '0845'
  elif (df["time"] >= "09:00:00") and (df["time"] < "09:15:00"):
    return '0900'
  elif (df["time"] >= "09:15:00") and (df["time"] < "09:30:00"):
    return '0915'
  elif (df["time"] >= "09:30:00") and (df["time"] < "09:45:00"):
    return '0930'
  elif (df["time"] >= "09:45:00") and (df["time"] < "10:00:00"):
    return '0945'
  elif (df["time"] >= "10:00:00") and (df["time"] < "10:15:00"):
    return '1000'
  elif (df["time"] >= "10:15:00") and (df["time"] < "10:30:00"):
    return '1015'
  elif (df["time"] >= "10:30:00") and (df["time"] < "10:45:00"):
    return '1030'
  elif (df["time"] >= "10:45:00") and (df["time"] < "11:00:00"):
    return '1045'
  elif (df["time"] >= "11:00:00") and (df["time"] < "11:15:00"):
    return '1100'
  elif (df["time"] >= "11:15:00") and (df["time"] < "11:30:00"):
    return '1115'
  elif (df["time"] >= "11:30:00") and (df["time"] < "11:45:00"):
    return '1130'
  elif (df["time"] >= "11:45:00") and (df["time"] < "12:00:00"):
    return '1145'
  elif (df["time"] >= "12:00:00") and (df["time"] < "12:15:00"):
    return '1200'
  elif (df["time"] >= "12:15:00") and (df["time"] < "12:30:00"):
    return '1215'
  elif (df["time"] >= "12:30:00") and (df["time"] < "12:45:00"):
    return '1230'
  elif (df["time"] >= "12:45:00") and (df["time"] < "13:00:00"):
    return '1245'
  elif (df["time"] >= "13:00:00") and (df["time"] < "13:15:00"):
    return '1300'
  elif (df["time"] >= "13:15:00") and (df["time"] < "13:30:00"):
    return '1315'
  elif (df["time"] >= "13:30:00") and (df["time"] < "13:45:00"):
    return '1330'
  elif (df["time"] >= "13:45:00") and (df["time"] < "14:00:00"):
    return '1345'
  elif (df["time"] >= "14:00:00") and (df["time"] < "14:15:00"):
    return '1400'
  elif (df["time"] >= "14:15:00") and (df["time"] < "14:30:00"):
    return '1415'
  elif (df["time"] >= "14:30:00") and (df["time"] < "14:45:00"):
    return '1430'
  elif (df["time"] >= "14:45:00") and (df["time"] < "15:00:00"):
    return '1445'
  elif (df["time"] >= "15:00:00") and (df["time"] < "15:15:00"):
    return '1500'
  elif (df["time"] >= "15:15:00") and (df["time"] < "15:30:00"):
    return '1515'
  elif (df["time"] >= "15:30:00") and (df["time"] < "15:45:00"):
    return '1530'
  elif (df["time"] >= "15:45:00") and (df["time"] < "16:00:00"):
    return '1545'
  elif (df["time"] >= "16:00:00") and (df["time"] < "16:15:00"):
    return '1600'
  elif (df["time"] >= "16:15:00") and (df["time"] < "16:30:00"):
    return '1615'
  elif (df["time"] >= "16:30:00") and (df["time"] < "16:45:00"):
    return '1630'
  elif (df["time"] >= "16:45:00") and (df["time"] < "17:00:00"):
    return '1645'
  elif (df["time"] >= "17:00:00") and (df["time"] < "17:15:00"):
    return '1700'
  elif (df["time"] >= "17:15:00") and (df["time"] < "17:30:00"):
    return '1715'
  elif (df["time"] >= "17:30:00") and (df["time"] < "17:45:00"):
    return '1730'
  elif (df["time"] >= "17:45:00") and (df["time"] < "18:00:00"):
    return '1745'
  elif (df["time"] >= "18:00:00") and (df["time"] < "18:15:00"):
    return '1800'
  elif (df["time"] >= "18:15:00") and (df["time"] < "18:30:00"):
    return '1815'
  elif (df["time"] >= "18:30:00") and (df["time"] < "18:45:00"):
    return '1830'
  elif (df["time"] >= "18:45:00") and (df["time"] < "19:00:00"):
    return '1845'
  elif (df["time"] >= "19:00:00") and (df["time"] < "19:15:00"):
    return '1900'
  elif (df["time"] >= "19:15:00") and (df["time"] < "19:30:00"):
    return '1915'
  elif (df["time"] >= "19:30:00") and (df["time"] < "19:45:00"):
    return '1930'
  elif (df["time"] >= "19:45:00") and (df["time"] < "20:00:00"):
    return '1945'
  elif (df["time"] >= "20:00:00") and (df["time"] < "20:15:00"):
    return '2000'
  elif (df["time"] >= "20:15:00") and (df["time"] < "20:30:00"):
    return '2015'
  elif (df["time"] >= "20:30:00") and (df["time"] < "20:45:00"):
    return '2030'
  elif (df["time"] >= "20:45:00") and (df["time"] < "21:00:00"):
    return '2045'
  elif (df["time"] >= "21:00:00") and (df["time"] < "21:15:00"):
    return '2100'
  elif (df["time"] >= "21:15:00") and (df["time"] < "21:30:00"):
    return '2115'
  elif (df["time"] >= "21:30:00") and (df["time"] < "21:45:00"):
    return '2130'
  elif (df["time"] >= "21:45:00") and (df["time"] < "22:00:00"):
    return '2145'
  elif (df["time"] >= "22:00:00") and (df["time"] < "22:15:00"):
    return '2200'
  elif (df["time"] >= "22:15:00") and (df["time"] < "22:30:00"):
    return '2215'
  elif (df["time"] >= "22:30:00") and (df["time"] < "22:45:00"):
    return '2230'
  elif (df["time"] >= "22:45:00") and (df["time"] < "23:00:00"):
    return '2245'
  elif (df["time"] >= "23:00:00") and (df["time"] < "23:15:00"):
    return '2300'
  elif (df["time"] >= "23:15:00") and (df["time"] < "23:30:00"):
    return '2315'
  elif (df["time"] >= "23:30:00") and (df["time"] < "23:45:00"):
    return '2330'
  elif (df["time"] >= "23:45:00") and (df["time"] <= "23:59:59"):
    return '2345'
  else:
    return 'NA'

In [ ]:
def towregion(df): #assign tow region
  if df["sna"] == "YouBike2.0_臺大水源舍區A棟":
    return '範圍E區 水源校區', 'E'
  elif df["sna"] == "YouBike2.0_臺大卓越研究大樓":
    return '範圍E區 水源校區', 'E'
  elif df["sna"] == "YouBike2.0_臺大水源修齊會館":
    return '範圍E區 水源校區', 'E'
  elif df["sna"] == "YouBike2.0_臺大檔案展示館":
    return '範圍E區 水源校區', 'E'
  elif df["sna"] == "YouBike2.0_臺大水源舍區B棟":
    return '範圍E區 水源校區', 'E'
  elif df["sna"] == "YouBike2.0_臺大男八舍東側":
    return '男八舍', 'D'
  elif df["sna"] == "YouBike2.0_臺大禮賢樓東南側":
    return '範圍C區 禮賢樓(卓聯)', 'C'
  elif df["sna"] == "YouBike2.0_臺大農業陳列館北側":
    return '農業陳列館', 'A'
  elif df["sna"] == "YouBike2.0_臺大管理學院二館北側":
    return '範圍C區 管理學院', 'C'
  elif df["sna"] == "YouBike2.0_臺大土木系館":
    return '土木系館', 'A'
  elif df["sna"] == "YouBike2.0_臺大大一女舍北側":
    return '範圍C區 研一、大一女舍', 'C'
  elif df["sna"] == "YouBike2.0_臺大女九舍西南側":
    return '女八九宿舍', 'B'
  elif df["sna"] == "YouBike2.0_臺大小福樓東側":
    return '小福', 'A'
  elif df["sna"] == "YouBike2.0_臺大立體機車停車場":
    return 'Ｄ區其他', 'D'
  elif df["sna"] == "YouBike2.0_臺大工綜館南側":
    return '工綜館', 'B'
  elif df["sna"] == "YouBike2.0_臺大天文數學館南側":
    return '思亮館', 'A'
  elif df["sna"] == "YouBike2.0_臺大心理系館南側":
    return '心理系', 'B'
  elif df["sna"] == "YouBike2.0_臺大樂學館東側":
    return '樂學館', 'A'
  elif df["sna"] == "YouBike2.0_臺大農化新館西側":
    return '農化系', 'C'
  elif df["sna"] == "YouBike2.0_臺大五號館西側":
    return '範圍C區 行政大樓', 'C'
  elif df["sna"] == "YouBike2.0_臺大舊體育館西側":
    return '舊體育館', 'A'
  elif df["sna"] == "YouBike2.0_臺大共同教室北側":
    return '共同教室', 'C'
  elif df["sna"] == "YouBike2.0_臺大共同教室東南側":
    return "共同教室", 'C'
  elif df["sna"] == "YouBike2.0_臺大鹿鳴堂東側":
    return '鹿鳴廣場', 'C'
  elif df["sna"] == "YouBike2.0_臺大公館停車場西北側":
    return '範圍C區 管理學院', 'C'
  elif df["sna"] == "YouBike2.0_臺大第二行政大樓南側":
    return '範圍C區 行政大樓', 'C'
  elif df["sna"] == "YouBike2.0_臺大明達館機車停車場":
    return '明達館', 'D'
  elif df["sna"] == "YouBike2.0_臺大二號館":
    return '物理系館', 'C'
  elif df["sna"] == "YouBike2.0_臺大凝態館南側":
    return '新體育館', 'A'
  elif df["sna"] == "YouBike2.0_臺大社科院西側":
    return '社會系館', 'B'
  elif df["sna"] == "YouBike2.0_臺大社會系館南側":
    return '社會系館', 'B'
  elif df["sna"] == "YouBike2.0_臺大思亮館東南側":
    return '思亮館', 'A'
  elif df["sna"] == "YouBike2.0_臺大椰林小舖":
    return '範圍C區 研一、大一女舍', 'C'
  elif df["sna"] == "YouBike2.0_臺大計資中心南側":
    return '計算機中心', 'C'
  elif df["sna"] == "YouBike2.0_臺大原分所北側":
    return '原分所', 'A'
  elif df["sna"] == "YouBike2.0_臺大校史館南側":
    return '文學院', 'A'
  elif df["sna"] == "YouBike2.0_臺大生命科學館西北側":
    return '範圍C區 生命科學館', 'C'
  elif df["sna"] == "YouBike2.0_臺大第一活動中心西南側":
    return '第一活動中心', 'B'
  elif df["sna"] == "YouBike2.0_臺大博理館西側":
    return '博理館', 'B'
  elif df["sna"] == "YouBike2.0_臺大博雅館西側":
    return '博雅館', 'A'
  elif df["sna"] == "YouBike2.0_臺大森林館北側":
    return '土木系館', 'A'
  elif df["sna"] == "YouBike2.0_臺大一號館":
    return '範圍C區 女一舍門口', 'C'
  elif df["sna"] == "YouBike2.0_臺大小小福西南側":
    return '小小福', 'C'
  elif df["sna"] == "YouBike2.0_臺大教研館北側":
    return '範圍C區 管理學院教研館', 'C'
  elif df["sna"] == "YouBike2.0_臺大四號館東北側":
    return '土木系館', 'A'
  elif df["sna"] == "YouBike2.0_臺大新生教室南側":
    return '新生大樓', 'A'
  elif df["sna"] == "YouBike2.0_臺大鄭江樓北側":
    return '鄭江樓', 'B'
  elif df["sna"] == "YouBike2.0_臺大電機二館東南側":
    return '電機二館', 'B'
  elif df["sna"] == "YouBike2.0_臺大圖資系館北側":
    return '圖資系', 'B'
  elif df["sna"] == "YouBike2.0_臺大總圖書館西南側":
    return '總圖書館', 'B'
  elif df["sna"] == "YouBike2.0_臺大黑森林西側":
    return '霖澤館', 'B'
  elif df["sna"] == "YouBike2.0_臺大獸醫館南側":
    return '獸醫系館', 'B'
  elif df["sna"] == "YouBike2.0_臺大新體育館東南側":
    return '新體育館', 'A'
  else:
    return 'NA'

#### 處理拖吊資料

In [ ]:
TowFiles = glob("../Input/towOfficial/towOfficial_*.csv")
i = 0
TowOfficial = []
for f in TowFiles:
  if i == 2:
    TowOfficial.append(pd.read_csv(f, header = None, names = ["違規區域", "拖吊日期", "範圍"]))
  else:
    TowOfficial.append(pd.read_csv(f))
  i += 1
TowOfficial = pd.concat(TowOfficial, ignore_index=True)

,違規區域,拖吊日期,範圍
0,土木系館,2019-04-19 15:20,範圍A區
1,土木系館,2019-04-19 15:20,範圍A區
2,土木系館,2019-03-14 15:20,範圍A區
3,土木系館,2019-03-14 15:20,範圍A區
4,土木系館,2019-03-14 15:20,範圍A區
...,...,...,...
34039,Ｄ區其他,2021-04-22 15:37,範圍D區
34040,Ｄ區其他,2021-04-22 15:38,範圍D區
34041,Ｄ區其他,2021-04-22 15:38,範圍D區
34042,Ｄ區其他,2021-04-28 15:11,範圍D區


In [ ]:
TowOfficial = TowOfficial.rename(columns = {'違規區域':'place', '拖吊日期':'datetime', '範圍':'region'})
TowOfficial.drop(TowOfficial.loc[TowOfficial['datetime'] == '拖吊日期'].index, inplace=True)
TowOfficial['region'] = TowOfficial['region'].str[2]
TowOfficial['datetime'] = pd.to_datetime(TowOfficial['datetime']).dt.strftime('%Y/%m/%d %H:%M:%S')
TowOfficial['time'] = pd.to_datetime(TowOfficial['datetime']).dt.strftime('%H:%M:%S')
TowOfficial['date'] = pd.to_datetime(TowOfficial['datetime']).dt.strftime('%Y/%m/%d')
TowOfficial['timerange'] = TowOfficial.apply(timerange, axis = 1)
TowOfficial = TowOfficial[['time', 'region', 'place', 'date', 'datetime', 'timerange']]

In [ ]:
TowOfficial.to_csv('../Output/TowOfficial.csv', encoding='utf-8', index = False) #save to csv

In [ ]:
TowFiles = glob("../Input/towData/towData_*.csv")
TowData = pd.concat([pd.read_csv(f).iloc[:,0:5] for f in TowFiles])

In [ ]:
TowData["time"] = pd.to_datetime(TowData["time"] + " " + TowData["AM/PM"]).dt.strftime('%H:%M:%S') #convert 12h to 24h
TowData["datetime"] = pd.to_datetime(TowData["date"] + " " + TowData["time"]) #create datetime
TowData = TowData.drop('AM/PM', 1) #delete column:["AM/PM"]

In [ ]:
TowData['timerange'] = TowData.apply(timerange, axis = 1)

In [ ]:
TowData['region'] = np.where(TowData.place.str.contains('範圍'), TowData.place.str[2], TowData.region)

In [ ]:
TowCountPlace = TowData.groupby(["place","date","timerange"]) #count
TowCountPlace = TowCountPlace.size().reset_index(name='countPlace')
TowCountPlace.to_csv('../Output/TowCountPlace.csv', encoding='utf-8', index = False) #save to csv

In [ ]:
TowCountRegion = TowData.groupby(["region","date","timerange"]) #count
TowCountRegion = TowCountRegion.size().reset_index(name='countRegion')
TowCountRegion.to_csv('../Output/TowCountRegion.csv', encoding='utf-8', index = False) #save to csv

In [ ]:
TowData.to_csv('../Output/TowData.csv', encoding='utf-8', index = False) #save to csv

#### 處理Ubike資料

In [ ]:
#UbikeFiles = glob("../Input/ubikeData/unread/ubikeData_20211121/ubikeData_20211121_*.csv")
UbikeFiles = glob("../Input/ubikeData/unread/ubikeData_*/ubikeData_*_*.csv")
UbikeData = []
for f in UbikeFiles:
    base = os.path.basename(f)
    basedate = os.path.splitext(base)[0][10:18]
    baserange = os.path.splitext(base)[0][-4:]
    
    data = pd.read_csv(f).iloc[:,[0,1,2,3,13]]
    data["date"] = datetime.strptime(basedate, '%Y%m%d').strftime('%Y/%m/%d')
    data["timerange"] = baserange
    data["weekday"] = datetime.strptime(basedate, '%Y%m%d').strftime('%w')
    
    UbikeData.append(data)
    
UbikeData = pd.concat(UbikeData)

In [ ]:
UbikeData['temp'] = UbikeData.apply(towregion, axis = 1)
UbikeData['place'] = [d[0] for d in UbikeData.temp]
UbikeData['region'] = [d[1] for d in UbikeData.temp]
UbikeData = UbikeData.drop('temp', 1) #delete column:["temp"]

In [ ]:
UbikeData = UbikeData.drop('sno', 1)

In [ ]:
#move unread data to read data folder
source_dir = '../Input/ubikeData/unread/'
target_dir = '../Input/ubikeData/read/'
    
file_names = os.listdir(source_dir)
    
for file_name in file_names:
    shutil.move(os.path.join(source_dir, file_name), target_dir)

UbikeData["mday"] = pd.to_datetime(UbikeData["mday"]) #convert type
UbikeData = UbikeData.rename(columns={'mday': 'datetime'}) #rename "mday" to "datetime"
UbikeData["date"] = pd.to_datetime(UbikeData["datetime"]).dt.date #get date
UbikeData["time"] = pd.to_datetime(UbikeData["datetime"]).dt.time #get time

#### 合併資料

In [ ]:
UbikeData['timerange'] = UbikeData['timerange'].astype(str) 
Bike = pd.merge(UbikeData, TowCountPlace, on=['date','timerange','place'], how='left')
Bike = pd.merge(Bike, TowCountRegion, on=['date','timerange','region'], how='left')

In [ ]:
Bike = Bike.fillna(0)

In [ ]:
Bike.to_csv('../Output/Bike.csv', encoding='utf-8', index = False) #save first df to csv